# This file automates the comparison between homogenous and heterogenous timesteps

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math

In [2]:
"""
Load all files from a given folder into a dictionary.
For CSV files, the content is loaded as a pandas DataFrame.
For other files, the content is read as plain text.
The keys of the dictionary are filenames without extensions.
"""
def load_files(folder_path):
    files_data = {}
    
    # List all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a subdirectory)
        if os.path.isfile(file_path):
            # Remove file extension from the key name
            file_key = os.path.splitext(filename)[0]
            
            # If the file is a CSV, load it as a DataFrame
            if filename.endswith('.csv'):
                files_data[file_key] = pd.read_csv(file_path)
            else:
                # For non-CSV files, read as plain text
                with open(file_path, 'r', encoding='utf-8') as file:
                    files_data[file_key] = file.read()
    
    return files_data

def add_row(df, row_type, total_homogenous, total_heterogeneous):
    # Calculate absolute and percentage difference
    absolute_difference = total_homogenous - total_heterogeneous
    percentage_difference = (absolute_difference / total_homogenous) * 100 if total_homogenous != 0 else 0

    # Create a new row as a DataFrame with each value in a list
    new_row = pd.DataFrame({
        'type': [row_type],
        'total_homogenous': [total_homogenous],
        'total_heterogeneous': [total_heterogeneous],
        'absolute_difference': [absolute_difference],
        'percentage_decrease (HO->HE) (%)': [percentage_difference]
    })

    # Concatenate the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df


In [3]:
####################### NEEED TO UPDATE MANUALLY PATH #######################

#Step 1: Load results  
heterogenous_results = load_files(r"C:\Users\Diego\GenX\GenX.jl-main\test_systems\3_simple_1week_easiest\results_HE")
homogenous_results = load_files(r"C:\Users\Diego\GenX\GenX.jl-main\test_systems\3_simple_1week_easiest\results_HO")

In [4]:
#Step 2: Create the dataframe containing major total results
data = {
    'type': [],
    'total_homogenous': [],
    'total_heterogeneous': [],
    'absolute_difference': [],
    'percentage_decrease (HO->HE) (%)': []
}
total_results = pd.DataFrame(data)

In [5]:
#Step 3: Load the major total results from their files

#Capacity
for (index_ho, row_ho), (index_he, row_he) in zip(
    homogenous_results['capacity'].loc[:, ['Resource', 'NewCap']].iterrows(), 
    heterogenous_results['capacity'].loc[:, ['Resource', 'NewCap']].iterrows()
):
    # Check if at least one of the 'Total' values is greater than zero
    if row_ho['NewCap'] > 0 or row_he['NewCap'] > 0:
        # Add the results to total_results
        total_results = add_row(total_results, f"Installed Capacity [MW]: {row_ho['Resource']}", row_ho['NewCap'], row_he['NewCap'])
                                
# Costs
for (index_ho, row_ho), (index_he, row_he) in zip(
    homogenous_results['costs'].loc[:, 'Costs':'Total'].iterrows(), 
    heterogenous_results['costs'].loc[:, 'Costs':'Total'].iterrows()
):
    # Check if at least one of the 'Total' values is greater than zero
    if row_ho['Total'] > 0 or row_he['Total'] > 0:
        # Add the results to total_results
        total_results = add_row(total_results, row_ho['Costs'], row_ho['Total'], row_he['Total'])


#Power 
homogenous_annual_sum = homogenous_results['power'].loc[homogenous_results['power']['Resource'] == 'AnnualSum'].squeeze()
heterogenous_annual_sum = heterogenous_results['power'].loc[heterogenous_results['power']['Resource'] == 'AnnualSum'].squeeze()

# Iterate over columns in homogenous and heterogenous data
for i, j in zip(homogenous_annual_sum.index[1:], heterogenous_annual_sum.index[1:]):  # Skip the first column (assumed to be 'Resource')
    homogenous_value = homogenous_annual_sum[i]
    heterogenous_value = heterogenous_annual_sum[j]
    
    # Check values and add row if both are greater than zero
    if homogenous_value > 0 or heterogenous_value > 0:
        total_results = add_row(total_results, f"Power [MWh]: {j}", homogenous_value, heterogenous_value)



In [6]:
total_results

,type,total_homogenous,total_heterogeneous,absolute_difference,percentage_decrease (HO->HE) (%)
0,Installed Capacity [MW]: CT_natural_gas_combin...,1.277200e+04,1.277200e+04,0.000000e+00,0.000000e+00
1,Installed Capacity [MW]: Total,1.277200e+04,1.277200e+04,0.000000e+00,0.000000e+00
2,cTotal,1.028692e+09,1.028692e+09,4.768372e-07,4.635372e-14
3,cFix,9.591517e+08,9.591517e+08,0.000000e+00,0.000000e+00
4,cVar,5.858777e+06,5.858777e+06,9.313226e-10,1.589619e-14
5,cFuel,6.368179e+07,6.368179e+07,0.000000e+00,0.000000e+00
6,Power [MWh]: CT_natural_gas_combined_cycle,1.641114e+06,1.641114e+06,0.000000e+00,0.000000e+00
7,Power [MWh]: Total,1.641114e+06,1.641114e+06,0.000000e+00,0.000000e+00
